In [112]:
import os
import re
import ast
import json
import re
from typing import Union
from langchain.schema import AIMessage  # Assuming you're using langchain

from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from dotenv import load_dotenv

In [113]:
load_dotenv()
## load the GROQ API Key
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

## If you do not have open AI key use the below Huggingface embedding
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [114]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# llm = ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")
llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama-3.1-8b-instant")

In [ ]:
llm.invoke("tell me a joke")

In [ ]:
embeddings.embed_query("what is the embeddigng of this sentence?")